In [ ]:
! apt install aria2

In [ ]:
! aria2c 'magnet:?xt=urn:btih:38d9ed996a5a75a039b84cf8a137be794e7cee89&tr=https%3A%2F%2Facademictorrents.com%2Fannounce.php&tr=udp%3A%2F%2Ftracker.coppersurfer.tk%3A6969&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337%2Fannounce'

In [ ]:
! mkdir hockey
! mv HockeyFights.zip hockey
! cd hockey && unzip HockeyFights.zip

In [ ]:
! mkdir -p ./Hockey_fights/{Dataframes,Violence,Non-violence}

In [ ]:
! mv hockey/no* ./Hockey_fights/Violence/
! mv hockey/fi* ./Hockey_fights/Non-violence/

In [ ]:
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model, layers
from random import shuffle
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow import keras

In [ ]:
NUM_FRAME = 10
IMG_SIZE = 100

LR = 0.0001
VIOLENCE_DIR = './Hockey_fights/Violence/'
NON_VIOLENCE_DIR = './Hockey_fights/Non-violence/'


Extracting Dataframes

In [ ]:
count = 0
def extract_frames(folder):
    count = 0
    for files in os.listdir(folder):
        cap = cv2.VideoCapture(os.path.join(folder, files))
        sucess, image = cap.read()
        sucess = True
        while sucess:
            sucess, image = cap.read()
            cv2.imwrite("./Hockey_fights/Dataframes/"+ str(count) + ".jpg", image)

            if cv2.waitKey(10) == 27:
                break
            count += 1

In [ ]:
count = 0
for files in os.listdir(VIOLENCE_DIR):
    cap = cv2.VideoCapture(os.path.join(VIOLENCE_DIR, files))
    sucess, image = cap.read()
    #print(sucess)
    sucess = True
    while sucess:
        sucess, image = cap.read()
        if not sucess:
            break
        cv2.imwrite("./Hockey_fights/Dataframes/"+ str(count) + ".jpg", image)

        if cv2.waitKey(10) == 27:
            break
        count += 1

In [ ]:
for files in os.listdir(NON_VIOLENCE_DIR):
    cap = cv2.VideoCapture(os.path.join(NON_VIOLENCE_DIR, files))
    sucess, image = cap.read()
    #print(sucess)
    sucess = True
    while sucess:
        sucess, image = cap.read()
        if not sucess:
            break
        cv2.imwrite("./Hockey_fights/Dataframes/"+ str(count) + ".jpg", image)

        if cv2.waitKey(10) == 27:
            break
        count += 1

In [ ]:
print(count)

40056


In [ ]:
def create_dataset():
    dataset = []
    image = []
    limit = 0
    c = 0

    for file in tqdm(os.listdir('./Hockey_fights/Dataframes/')):
        path = os.path.join('./Hockey_fights/Dataframes/', file)
        img = cv2.resize(cv2.imread(path), (IMG_SIZE, IMG_SIZE))
        image.append(np.array(img))
        limit += 1
        c += 1
        if c == NUM_FRAME:
            c = 0
            if limit < 20057:
                dataset.append(np.array([image, np.array([1, 0])], dtype=object))
            elif limit >= 20056:
                dataset.append(np.array([image, np.array([0, 1])], dtype=object))
            image = []

    shuffle(dataset)
    np.save('dataset.npy', dataset)
    return dataset

In [ ]:
data = create_dataset()

100%|██████████| 40056/40056 [01:09<00:00, 572.59it/s]


In [ ]:
data

In [ ]:
train = data[:-400]
test = data[-400:]

Creating the Model

In [ ]:
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, TimeDistributed,Dropout, Activation, Flatten,Conv2D, MaxPooling2D,LSTM,Bidirectional, Reshape
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import Input

inputs = Input(shape=[10, IMG_SIZE, IMG_SIZE, 3], name= "input")
convrnn = Conv2D(64, (3, 3), activation='relu', padding = 'same')(inputs)
convrnn = Conv2D(64, (3, 3), activation='relu')(convrnn)
print(convrnn[0][0].shape)
convrnn = MaxPooling2D((2,2))(convrnn[0])
print(convrnn.shape)
convrnn = Conv2D(64, (3, 3), activation='relu')(convrnn)
print(convrnn.shape)
convrnn = MaxPooling2D((2,2))(convrnn)
print(convrnn.shape)
convrnn = Conv2D(64, (3, 3), activation='relu')(convrnn)
convrnn = MaxPooling2D((2,2))(convrnn)
print(convrnn.shape)
convrnn = Flatten()(convrnn)
print(convrnn.shape)

convrnn = tf.reshape(convrnn, (1, 10, 6400))
# print(convrnn.shape)

lstm_fw = LSTM(units=32)
lstm_bw = LSTM(units=32, go_backwards=True)
bi_lstm = Bidirectional(lstm_fw, backward_layer=lstm_bw)(convrnn)

convrnn = Dense(64, activation= 'relu')(bi_lstm)
convrnn = Dense(32, activation= 'relu')(convrnn)
output = Dense(1, activation='sigmoid')(convrnn)

model = tf.keras.Model(inputs = inputs, outputs = output)
model.summary()

(98, 98, 64)
(10, 49, 49, 64)
(10, 47, 47, 64)
(10, 23, 23, 64)
(10, 10, 10, 64)
(10, 6400)
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 10, 100, 100, 3   0         
                             )]                                  
                                                                 
 conv2d (Conv2D)             (None, 10, 100, 100, 64   1792      
                             )                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 10, 98, 98, 64)    36928     
                                                                 
 tf.__operators__.getitem_2  (10, 98, 98, 64)          0         
  (SlicingOpLambda)                                              
                                                                 
 max_pooling2d (MaxPooling2  (10, 4

In [ ]:
len(train[0][1])

2

In [ ]:
from tensorflow.keras.layers import LSTM, Bidirectional, Conv2D, Dense, Flatten, MaxPooling2D, TimeDistributed, Reshape, MaxPooling3D
from tensorflow.keras.models import Sequential

cnn = Sequential()
cnn.add(Conv2D(64, (3, 3), activation='relu', input_shape=(10, IMG_SIZE, IMG_SIZE, 3), padding="same"))
cnn.add(Conv2D(64, (3, 3), activation='relu'))
cnn.add(MaxPooling3D((1, 2,2)))
cnn.add(Conv2D(64, (3, 3), activation='relu'))
cnn.add(MaxPooling3D((1, 2,2)))
cnn.add(Conv2D(64, (3, 3), activation='relu'))
cnn.add(MaxPooling3D((1, 2,2)))
#cnn.add(Flatten())
cnn.add(Reshape((10, 6400)))

#cnn.summary()

lstm_fw = LSTM(units=32)
lstm_bw = LSTM(units=32, go_backwards = True)
model = Sequential()
#model.add(TimeDistributed(cnn, input_shape=(1, 10, 10, 64)))
model.add(Bidirectional(lstm_fw, backward_layer = lstm_bw))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

cnn.add(Bidirectional(lstm_fw, backward_layer = lstm_bw))
cnn.add(Dense(64, activation='relu'))
cnn.add(Dense(32, activation='relu'))
cnn.add(Dense(2, activation='sigmoid'))

cnn.summary()
opt = keras.optimizers.Adam(learning_rate=LR)
cnn.compile(loss='categorical_crossentropy',optimizer='SGD',metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 10, 100, 100, 64   1792      
                             )                                   
                                                                 
 conv2d_5 (Conv2D)           (None, 10, 98, 98, 64)    36928     
                                                                 
 max_pooling3d (MaxPooling3  (None, 10, 49, 49, 64)    0         
 D)                                                              
                                                                 
 conv2d_6 (Conv2D)           (None, 10, 47, 47, 64)    36928     
                                                                 
 max_pooling3d_1 (MaxPoolin  (None, 10, 23, 23, 64)    0         
 g3D)                                                            
                                                        

In [ ]:
data = np.load('dataset.npy', allow_pickle=True)

In [ ]:
train, test = train_test_split(data, train_size= 0.9, shuffle=True)


In [ ]:
train[0][1]

array([0, 1])

In [ ]:
X = np.array([i[0] for i in train]).reshape(-1, 10, IMG_SIZE, IMG_SIZE, 3)
Y = np.array([i[1] for i in train])

x_valid = np.array([i[0] for i in test]).reshape(-1, 10, IMG_SIZE, IMG_SIZE, 3)
y_valid = np.array([i[1] for i in test])

In [ ]:

X = X.astype('float32')/255
x_valid = x_valid.astype('float32')/255
#Y = Y.astype('float32').reshape((-1, 1))
#y_valid = y_valid.astype('float32').reshape((-1, 1))



In [ ]:
print(Y.shape)
print(y_valid.shape)
print(X.shape)
print(x_valid.shape)

print(y_valid[7])

(3604, 2)
(401, 2)
(3604, 10, 100, 100, 3)
(401, 10, 100, 100, 3)
[0 1]


In [ ]:
x_try = X[:20]
y_try = Y[:20]
print(len(x_try), len(y_try))

20 20


In [ ]:
cnn.fit(X, Y, epochs=10, validation_data=(x_valid,y_valid), batch_size=16, verbose=1)

In [ ]:
cnn.save('CNN-BiLSTM.h5', overwrite=True, include_optimizer=True)

In [ ]:
! du -sh *

3.2G	CNN-BiLSTM-Model
1.2G	dataset.npy
164M	hockey
1.3G	Hockey_fights
55M	sample_data
